# Concise Implementation of Recurrent Neural Networks
:label:`sec_rnn-concise`

While :numref:`sec_rnn_scratch` was instructive to see how RNNs are implemented,
this is not convenient or fast.
This section will show how to implement the same language model more efficiently
using functions provided by high-level APIs
of a deep learning framework.
We begin as before by reading the time machine dataset.


In [ ]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
    
%maven ai.djl:api:0.11.0-SNAPSHOT
%maven ai.djl:basicdataset:0.11.0-SNAPSHOT

// See https://github.com/awslabs/djl/blob/master/mxnet/mxnet-engine/README.md
// MXNet 
%maven ai.djl.mxnet:mxnet-engine:0.11.0-SNAPSHOT
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-backport

In [ ]:
%load ../utils/plot-utils
// %load ../utils/Functions.java
%load ../utils/PlotUtils.java
// %load ../utils/TimeMachineUtils.java
// %load ../utils/StopWatch.java
// %load ../utils/Accumulator.java
%load ../utils/Animator.java
// %load ../utils/Training.java

%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/Functions.java
// %load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/PlotUtils.java
%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/TimeMachine.java
%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/StopWatch.java
%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/Accumulator.java
// %load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/Animator.java
%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/Training.java
%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/RNNModel.java
%load /Users/nivesmn/Documents/projects/intellij_notebooks/section8_6/src/main/java/RNNModelScratch.java

In [ ]:
import ai.djl.Device;
import ai.djl.ndarray.NDArray;
import ai.djl.ndarray.NDList;
import ai.djl.ndarray.NDManager;
import ai.djl.ndarray.types.DataType;
import ai.djl.ndarray.types.Shape;
import ai.djl.nn.recurrent.RNN;
import ai.djl.nn.AbstractBlock;
import ai.djl.nn.core.Linear;
import ai.djl.training.ParameterStore;
import ai.djl.util.Pair;
import ai.djl.util.PairList;

import java.io.IOException;
import java.util.ArrayList;

In [ ]:
NDManager manager = NDManager.newBaseManager(Functions.tryGpu(0));

In [ ]:
int batchSize = 32;
int numSteps = 35;

Pair<ArrayList<NDList>, Vocab> timeMachine =
        loadDataTimeMachine(batchSize, numSteps, false, 10000);
ArrayList<NDList> trainIter = timeMachine.getKey();
Vocab vocab = timeMachine.getValue();

## Defining the Model

High-level APIs provide implementations of recurrent neural networks.
We construct the recurrent neural network layer `rnn_layer` with a single hidden layer and 256 hidden units.
In fact, we have not even discussed yet what it means to have multiple layers---this will happen in :numref:`sec_deep_rnn`.
For now, suffice it to say that multiple layers simply amount to the output of one layer of RNN being used as the input for the next layer of RNN.


In [ ]:
int numHiddens = 256;
RNN rnnLayer = RNN.builder().setNumLayers(1)
        .setStateSize(numHiddens).optReturnState(true).optBatchFirst(false).build();

Initializing the hidden state is straightforward.
We invoke the member function `beginState` _(In DJL we don't have to run `beginState` to later specify the resulting state the first time we run `forward`, as this logic is ran by DJL the first time we do `forward` but we will create it here for demonstration purposes)_.
This returns a list (`state`)
that contains
an initial hidden state
for each example in the minibatch,
whose shape is
(number of hidden layers, batch size, number of hidden units).
For some models 
to be introduced later 
(e.g., long short-term memory),
such a list also
contains other information.

In [ ]:
public static NDList beginState(int batchSize, int numLayers, int numHiddens) {
    return new NDList(manager.zeros(new Shape(numLayers, batchSize, numHiddens)));
}

NDList state = beginState(batchSize, 1, numHiddens);
System.out.println(state.size());
System.out.println(state.get(0).getShape());

With a hidden state and an input,
we can compute the output with
the updated hidden state.
It should be emphasized that
the "output" (`Y`) of `rnnLayer`
does *not* involve computation of output layers:
it refers to 
the hidden state at *each* time step,
and they can be used as the input
to the subsequent output layer.

Besides,
the updated hidden state (`stateNew`) returned by `rnnLayer`
refers to the hidden state
at the *last* time step of the minibatch.
It can be used to initialize the 
hidden state for the next minibatch within an epoch
in sequential partitioning.
For multiple hidden layers,
the hidden state of each layer will be stored
in this variable (`stateNew`).
For some models 
to be introduced later 
(e.g., long short-term memory),
this variable also
contains other information.

In [ ]:
NDArray X = manager.randomUniform (0, 1,new Shape(numSteps, batchSize, vocab.length()));

NDList forwardOutput = rnnLayer.forward(new ParameterStore(manager, false), new NDList(X, state.get(0)), false);
NDArray Y = forwardOutput.get(0);
NDArray stateNew = forwardOutput.get(1);

System.out.println(Y.getShape());
System.out.println(stateNew.getShape());

Similar to :numref:`sec_rnn_scratch`,
we define an `RNNModel` class 
for a complete RNN model.
Note that `rnnLayer` only contains the hidden recurrent layers, we need to create a separate output layer.

In [ ]:
public class RNNModel extends AbstractBlock {

    private static final byte VERSION = 2;
    private RNN rnnLayer;
    private Linear dense;
    private int vocabSize;

    public RNNModel(RNN rnnLayer, int vocabSize) {
        super(VERSION);
        this.rnnLayer = rnnLayer;
        this.addChildBlock("rnn", rnnLayer);
        this.vocabSize = vocabSize;
        this.dense = Linear.builder().setUnits(vocabSize).build();
    }

    
    @Override
    protected NDList forwardInternal(ParameterStore parameterStore, NDList inputs, boolean training, PairList<String, Object> params) {
        NDArray X = inputs.get(0).transpose().oneHot(this.vocabSize);
        inputs.set(0, X);
        NDList result = this.rnnLayer.forward(parameterStore, inputs, training);
        NDArray Y = result.get(0);
        NDArray state = result.get(1);

        int shapeLength = Y.getShape().getShape().length;
        NDList output = this.dense.forward(parameterStore, new NDList(Y
                .reshape(new Shape(-1, Y.getShape().get(shapeLength-1)))), training);
        return new NDList(output.get(0), state);
    }
    
    @Override
    public void initializeChildBlocks(NDManager manager, DataType dataType, Shape... inputShapes) {
        /* rnnLayer is already initialized so we don't have to do anything here, just override it.*/
        return;
    }

    /* We won't implement this since we won't be using it but it's required as part of an AbstractBlock  */
    @Override
    public Shape[] getOutputShapes(Shape[] inputShapes) {
        return new Shape[0];
    }
}

## Training and Predicting

Before training the model, let us make a prediction with the a model that has random weights.


In [ ]:
Device device = Functions.tryGpu(0);
RNNModel net = new RNNModel(rnnLayer, vocab.length());
net.initialize(manager, DataType.FLOAT32, X.getShape());
String prediction = TimeMachine.predictCh8("time traveller", 10, net, vocab, device, manager);
System.out.println(prediction);

As is quite obvious, this model does not work at all. Next, we call `trainCh8` with the same hyperparameters defined in :numref:`sec_rnn_scratch` and train our model with high-level APIs.


In [ ]:
int numEpochs = 500;
int lr = 1;
TimeMachine.trainCh8((Object) net, trainIter, vocab, lr, numEpochs, device, false, manager);

Compared with the last section, this model achieves comparable perplexity,
albeit within a shorter period of time, due to the code being more optimized by
high-level APIs of the deep learning framework.


## Summary

* High-level APIs of the deep learning framework provides an implementation of the RNN layer.
* The RNN layer of high-level APIs returns an output and an updated hidden state, where the output does not involve output layer computation.
* Using high-level APIs leads to faster RNN training than using its implementation from scratch.

## Exercises

1. Can you make the RNN model overfit using the high-level APIs?
1. What happens if you increase the number of hidden layers in the RNN model? Can you make the model work?
1. Implement the autoregressive model of :numref:`sec_sequence` using an RNN.
